## Dropout Implementation 

What is dropout?
Dropout refers to data, or noise, that's intentionally dropped from a neural network to improve processing and time to results.
The human brain contains billions of neurons that fire electrical and chemical signals to each other to coordinate thoughts and life functions. A neural network uses a software equivalent of these neurons, called units. Each unit receives signals from other units and then computes an output that it passes onto other neuron/units, or nodes, in the network.
Go through this link to know more about **Dropout**

https://www.kaggle.com/discussions/general/397662

Required Libraries

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')

Load Dataset

In [19]:
df = pd.read_csv("../input/sonar-datasets/sonar_dataset.csv", header = None) # here I used header = None as this dataset doesn't have any header
df.sample(5)#it will show only 5 rows with all columns
#df # it will show all the rows and colums

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
11,0.0123,0.0309,0.0169,0.0313,0.0358,0.0102,0.0182,0.0579,0.1122,0.0835,...,0.0133,0.0265,0.0224,0.0074,0.0118,0.0026,0.0092,0.0009,0.0044,R
94,0.0025,0.0309,0.0171,0.0228,0.0434,0.1224,0.1947,0.1661,0.1368,0.1430,...,0.0149,0.0077,0.0036,0.0114,0.0085,0.0101,0.0016,0.0028,0.0014,R
125,0.0228,0.0853,0.1000,0.0428,0.1117,0.1651,0.1597,0.2116,0.3295,0.3517,...,0.0172,0.0191,0.0260,0.0140,0.0125,0.0116,0.0093,0.0012,0.0036,M
57,0.0216,0.0124,0.0174,0.0152,0.0608,0.1026,0.1139,0.0877,0.1160,0.0866,...,0.0052,0.0049,0.0096,0.0134,0.0122,0.0047,0.0018,0.0006,0.0023,R
79,0.0108,0.0086,0.0058,0.0460,0.0752,0.0887,0.1015,0.0494,0.0472,0.0393,...,0.0029,0.0078,0.0114,0.0083,0.0058,0.0003,0.0023,0.0026,0.0027,R


Data Exploration

The shape of a DataFrame is a tuple of array dimensions that tells the number of rows and columns of a given DataFrame.

In [20]:
df.shape

(208, 61)

Now I want to know if there is any column that containing null or not

The isna() method returns a DataFrame object where all the values are replaced with a Boolean value True for NA (not-a -number) values, and otherwise False.


In [22]:
df.isna()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
204,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
205,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
206,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


The sum() method adds all values in each column and returns the sum for each column

In [23]:
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

As there is no null values so I don't have to tackle the null values

In [24]:
df.columns #it just gives columns name. As we have no header& the columns name is only a neumerical values

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
           dtype='int64')

In [25]:
df[60].value_counts()

M    111
R     97
Name: 60, dtype: int64

The drop() method removes the specified row or column. By specifying the column axis ( axis='columns' ), the drop() method removes the specified column.

In [30]:
x = df.drop(60, axis = 'columns')
y = df[60]
x

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0203,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0051,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0155,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0042,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048


Look above, now the dataset doesn't have 60 columns

Now we will see y. Here y is not ready because it has data in categorical form like R , M. So we need to do one hot coding to ocnvert it into integer

In [31]:
y

0      R
1      R
2      R
3      R
4      R
      ..
203    M
204    M
205    M
206    M
207    M
Name: 60, Length: 208, dtype: object

As it has just two labels so I can do is just get dummies and then drop the first dummy. As I have two dummy columns then I can drop one.

In [32]:
y = pd.get_dummies(y, drop_first = True)
y.sample(5) # R ------> 1 & M -------> 0

,R
83,1
124,0
72,1
77,1
36,1


Here I have R columns which means 1 --> ROck & 0--> Metal.  Here we need only one column and that will be my target

In [34]:
y.value_counts()

R
0    111
1     97
dtype: int64

Here we can see my Rock and Metal finally converted into 0 and 1. Now my x & y are ready.

So the next step in Machine learning is divide data into train and test dataset. I am using test size of 25 percent. I use random state for reproducibility and if we are running this again and again, if we specifiy random state it will divided into similar type of splits.

In [38]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=1)

In [40]:
x_train.shape, x_test.shape

((156, 60), (52, 60))

In [42]:
x_test

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
186,0.0209,0.0191,0.0411,0.0321,0.0698,0.1579,0.1438,0.1402,0.3048,0.3914,...,0.0054,0.0078,0.0201,0.0104,0.0039,0.0031,0.0062,0.0087,0.0070,0.0042
155,0.0211,0.0128,0.0015,0.0450,0.0711,0.1563,0.1518,0.1206,0.1666,0.1345,...,0.0174,0.0117,0.0023,0.0047,0.0049,0.0031,0.0024,0.0039,0.0051,0.0015
165,0.0221,0.0065,0.0164,0.0487,0.0519,0.0849,0.0812,0.1833,0.2228,0.1810,...,0.0167,0.0089,0.0051,0.0015,0.0075,0.0058,0.0016,0.0070,0.0074,0.0038
200,0.0131,0.0387,0.0329,0.0078,0.0721,0.1341,0.1626,0.1902,0.2610,0.3193,...,0.0137,0.0150,0.0076,0.0032,0.0037,0.0071,0.0040,0.0009,0.0015,0.0085
58,0.0225,0.0019,0.0075,0.0097,0.0445,0.0906,0.0889,0.0655,0.1624,0.1452,...,0.0051,0.0034,0.0129,0.0100,0.0044,0.0057,0.0030,0.0035,0.0021,0.0027
34,0.0311,0.0491,0.0692,0.0831,0.0079,0.0200,0.0981,0.1016,0.2025,0.0767,...,0.0089,0.0087,0.0032,0.0130,0.0188,0.0101,0.0229,0.0182,0.0046,0.0038
151,0.0231,0.0315,0.0170,0.0226,0.0410,0.0116,0.0223,0.0805,0.2365,0.2461,...,0.0151,0.0125,0.0036,0.0123,0.0043,0.0114,0.0052,0.0091,0.0008,0.0092
18,0.0270,0.0092,0.0145,0.0278,0.0412,0.0757,0.1026,0.1138,0.0794,0.1520,...,0.0045,0.0084,0.0010,0.0018,0.0068,0.0039,0.0120,0.0132,0.0070,0.0088
202,0.0272,0.0378,0.0488,0.0848,0.1127,0.1103,0.1349,0.2337,0.3113,0.3997,...,0.0146,0.0091,0.0045,0.0043,0.0043,0.0098,0.0054,0.0051,0.0065,0.0103
62,0.0086,0.0215,0.0242,0.0445,0.0667,0.0771,0.0499,0.0906,0.1229,0.1185,...,0.0047,0.0072,0.0054,0.0022,0.0016,0.0029,0.0058,0.0050,0.0024,0.0030


# Using Deep Learning Model

### Model without dropout layer

In [44]:
model = keras.Sequential([
    keras.layers.Dense(60, input_dim = 60, activation = 'relu'), #as we have 60 neurons in input layer(for 60 columns in total) and they are connecting to hidden layer which also have 60 neuron
     #will create 2 more hidden layers
    keras.layers.Dense(30, activation = 'relu'),
    keras.layers.Dense(15, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid') #this is a output layer where I have 1 neuron because of binary classification problem
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics =['accuracy'])
model.fit(x_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 4ms/step - loss: 0.6866 - accuracy: 0.5897
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6653 - accuracy: 0.6474
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6422 - accuracy: 0.6795
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6185 - accuracy: 0.7179
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5947 - accuracy: 0.7051
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5804 - accuracy: 0.7179
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5270 - accuracy: 0.7308
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4857 - accuracy: 0.7949
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4532 - accuracy: 0.8205
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4335 - accuracy: 0.8205
Epoch 11/

In [45]:
model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 6ms/step - loss: 1.1544 - accuracy: 0.7308


[1.1543891429901123, 0.7307692170143127]

In [46]:
y_pred = model.predict(x_test).reshape(-1)
print(y_pred[:10])

#Sigmoid predicts 0 & 1 so I need to round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 4ms/step
[4.7849943e-07 9.0081114e-01 8.4761119e-01 1.2867355e-05 9.9999976e-01
 9.9995506e-01 2.4774756e-02 9.9999833e-01 1.7322011e-05 9.9999988e-01]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [47]:
y_test[:10]

,R
186,0
155,0
165,0
200,0
58,1
34,1
151,0
18,1
202,0
62,1


In [48]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.85      0.77        27
           1       0.79      0.60      0.68        25

    accuracy                           0.73        52
   macro avg       0.74      0.73      0.72        52
weighted avg       0.74      0.73      0.73        52



### Model with Dropout Layer

In [49]:
modeld = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid') #this is a output layer where I have 1 neuron because of binary classification problem
])

modeld.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modeld.fit(x_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 2s 4ms/step - loss: 0.7487 - accuracy: 0.5256
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6938 - accuracy: 0.5641
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6877 - accuracy: 0.5192
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.7054 - accuracy: 0.5577
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.7141 - accuracy: 0.5385
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.5513
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7012 - accuracy: 0.5513
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6929 - accuracy: 0.5321
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6753 - accuracy: 0.6154
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7050 - accuracy: 0.5192
Epoch 11/

In [50]:
modeld.evaluate(x_test, y_test)

2/2 [==============================] - 0s 8ms/step - loss: 0.4505 - accuracy: 0.7692


[0.45047956705093384, 0.7692307829856873]

Training Accuracy is still good but Test Accuracy Improved

In [51]:
y_pred = modeld.predict(x_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 4ms/step
[1.6750941e-04 8.1129688e-01 9.2020518e-01 2.4325792e-02 9.9400228e-01
 9.4838077e-01 3.3766431e-01 9.8964334e-01 1.5428182e-02 9.9607015e-01]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [52]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.81      0.79        27
           1       0.78      0.72      0.75        25

    accuracy                           0.77        52
   macro avg       0.77      0.77      0.77        52
weighted avg       0.77      0.77      0.77        52



Here accuracy is increased 0.73 to 0.77 after using  dropout technique.